<a href="https://colab.research.google.com/github/jbarrasa/goingmeta/blob/main/session5/Ontology_Driven_KG_creation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
pip install rdflib neo4j

In [ ]:
import rdflib, time

In [ ]:
#utility function to get the local part of a URI (stripping out the namespace)

def getLocalPart(uri):
  pos = -1
  pos = uri.rfind('#') 
  if pos < 0 :
    pos = uri.rfind('/')  
  if pos < 0 :
    pos = uri.rindex(':')
  return uri[pos+1:]

def getNamespacePart(uri):
  pos = -1
  pos = uri.rfind('#') 
  if pos < 0 :
    pos = uri.rfind('/')  
  if pos < 0 :
    pos = uri.rindex(':')
  return uri[0:pos+1]

# quick test
print(getLocalPart("http://onto.neo4j.com/rail#Station"))
print(getNamespacePart("http://onto.neo4j.com/rail#Station"))

In [ ]:
# processing the ontology...
 
g = rdflib.Graph()
g.parse("https://raw.githubusercontent.com/jbarrasa/goingmeta/main/session5/ontos/rail.ttl", format='turtle')

simple_query = """
prefix owl: <http://www.w3.org/2002/07/owl#> 
prefix rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#> 

SELECT DISTINCT ?c
  WHERE {
    ?c rdf:type owl:Class .    
  } """

for row in g.query(simple_query):
    print(str(row.c), getLocalPart(str(row.c)), getNamespacePart(str(row.c)))


In [ ]:
# read the onto and generate cypher (complete without mappings)

g = rdflib.Graph()
g.parse("https://raw.githubusercontent.com/jbarrasa/goingmeta/main/session5/ontos/rail.ttl", format='turtle')

classes_and_props_query = """
prefix owl: <http://www.w3.org/2002/07/owl#> 
prefix rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#> 

SELECT DISTINCT ?curi (GROUP_CONCAT(DISTINCT ?propTypePair ; SEPARATOR=",") AS ?props)
WHERE {
    ?curi rdf:type owl:Class .
    optional { 
      ?prop rdfs:domain ?curi ;
        a owl:DatatypeProperty ;
        rdfs:range ?range .
      BIND (concat(str(?prop),';',str(?range)) AS ?propTypePair)
    }
  } GROUP BY ?curi  """

cypher_list = []

for row in g.query(classes_and_props_query):
    cypher = []
    cypher.append("unwind $records AS record")
    cypher.append("merge (n:" + getLocalPart(row.curi) + " { `<id_prop>`: record.`<col with id>`} )")
    for pair in row.props.split(","):
      propName = pair.split(";")[0]
      propType = pair.split(";")[1]
      cypher.append("set n." + getLocalPart(propName) + " = record.`<col with value for " + getLocalPart(propName) + ">`")
    cypher.append("return count(*) as total") 
    cypher_list.append(' \n'.join(cypher))


rels_query = """
prefix owl: <http://www.w3.org/2002/07/owl#> 
prefix rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#> 

SELECT DISTINCT ?rel ?dom ?ran #(GROUP_CONCAT(DISTINCT ?relTriplet ; SEPARATOR=",") AS ?rels)
WHERE {
    ?rel a ?propertyClass .
    filter(?propertyClass in (rdf:Property, owl:ObjectProperty, owl:FunctionalProperty, owl:AsymmetricProperty, 
           owl:InverseFunctionalProperty, owl:IrreflexiveProperty, owl:ReflexiveProperty, owl:SymmetricProperty, owl:TransitiveProperty))
    
    ?rel rdfs:domain ?dom ;
      rdfs:range ?ran .
    
    #BIND (concat(str(?rel),';',str(?dom),';',str(?range)) AS ?relTriplet)
    
  }"""


for row in g.query(rels_query):
  cypher = []
  cypher.append("unwind $records AS record")
  cypher.append("match (source:" + getLocalPart(row.dom) + " { `<id_prop>`: record.`<col with source id>`} )")
  cypher.append("match (target:" + getLocalPart(row.ran) + " { `<id_prop>`: record.`<col with target id>`} )")
  cypher.append("merge (source)-[r:`"+ getLocalPart(row.rel) +"`]->(target)")
  cypher.append("return count(*) as total") 
  cypher_list.append(' \n'.join(cypher))

for q in cypher_list:
  print("\n\n" + q)


In [ ]:
railMappings = {}

stationMapping = {}
stationMapping["@fileName"] = "https://raw.githubusercontent.com/jbarrasa/goingmeta/main/session5/data/nr-stations-all.csv"
stationMapping["@uniqueId"] = "stationCode"
stationMapping["lat"] = "lat"
stationMapping["long"] = "long"
stationMapping["stationAddress"] = "address"
stationMapping["stationCode"] = "crs"
stationMapping["stationName"] = "name"
railMappings["Station"] = stationMapping

eventMapping = {}
eventMapping["@fileName"] = "https://raw.githubusercontent.com/jbarrasa/goingmeta/main/session5/data/nr-events.csv"
eventMapping["@uniqueId"] = "eventId"
eventMapping["eventDescription"] = "desc"
eventMapping["eventId"] = "id"
eventMapping["timestamp"] = "ts"
eventMapping["eventType"] = "type"
railMappings["Event"] = eventMapping

linkMapping = {}
linkMapping["@fileName"] = "https://raw.githubusercontent.com/jbarrasa/goingmeta/main/session5/data/nr-station-links.csv"
linkMapping["@from"] = "origin"
linkMapping["@to"] = "destination"
railMappings["link"] = linkMapping

affectsMapping = {}
affectsMapping["@fileName"] = "https://raw.githubusercontent.com/jbarrasa/goingmeta/main/session5/data/nr-events.csv"
affectsMapping["@from"] = "id"
affectsMapping["@to"] = "Station"
railMappings["affects"] = affectsMapping

# show it?
railMappings

In [ ]:
#copy of previous but using the mappings
def getLoadersFromOnto(onto, rdf_format, mappings):
  g = rdflib.Graph()
  g.parse(onto, format= rdf_format)

  classes_and_props_query = """
  prefix owl: <http://www.w3.org/2002/07/owl#> 
  prefix rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#> 

  SELECT DISTINCT ?curi (GROUP_CONCAT(DISTINCT ?propTypePair ; SEPARATOR=",") AS ?props)
  WHERE {
      ?curi rdf:type owl:Class .
      optional { 
        ?prop rdfs:domain ?curi ;
          a owl:DatatypeProperty ;
          rdfs:range ?range .
        BIND (concat(str(?prop),';',str(?range)) AS ?propTypePair)
      }
    } GROUP BY ?curi  """

  cypher_import = {}
  export_ns = set()
  export_mappings = {}

  for row in g.query(classes_and_props_query):
      export_ns.add(getNamespacePart(row.curi))
      export_mappings[getLocalPart(row.curi)] = str(row.curi)
      cypher = []
      cypher.append("unwind $records AS record")
      cypher.append("merge (n:" + getLocalPart(row.curi) + " { `" + mappings[getLocalPart(row.curi)]["@uniqueId"] + "`: record.`" + mappings[getLocalPart(row.curi)][mappings[getLocalPart(row.curi)]["@uniqueId"]] + "`} )")
      for pair in row.props.split(","):      
        propName = pair.split(";")[0]
        propType = pair.split(";")[1]
        export_ns.add(getNamespacePart(propName))
        export_mappings[getLocalPart(propName)] = propName
        #if a mapping (a column in the source file) is defined for the property and property is not a unique id
        if getLocalPart(propName) in mappings[getLocalPart(row.curi)] and getLocalPart(propName) != mappings[getLocalPart(row.curi)]["@uniqueId"]:
          cypher.append("set n." + getLocalPart(propName) + " = record.`" + mappings[getLocalPart(row.curi)][getLocalPart(propName)] + "`")
      cypher.append("return count(*) as total") 
      cypher_import[getLocalPart(row.curi)] = ' \n'.join(cypher)


  rels_query = """
  prefix owl: <http://www.w3.org/2002/07/owl#> 
  prefix rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#> 

  SELECT DISTINCT ?rel ?dom ?ran #(GROUP_CONCAT(DISTINCT ?relTriplet ; SEPARATOR=",") AS ?rels)
  WHERE {
      ?rel a ?propertyClass .
      filter(?propertyClass in (rdf:Property, owl:ObjectProperty, owl:FunctionalProperty, owl:AsymmetricProperty, 
            owl:InverseFunctionalProperty, owl:IrreflexiveProperty, owl:ReflexiveProperty, owl:SymmetricProperty, owl:TransitiveProperty))
      
      ?rel rdfs:domain ?dom ;
        rdfs:range ?ran .
      
      #BIND (concat(str(?rel),';',str(?dom),';',str(?range)) AS ?relTriplet)
      
    }"""

  for row in g.query(rels_query):
    export_ns.add(getNamespacePart(row.rel))
    export_mappings[getLocalPart(row.rel)] = str(row.rel)
    cypher = []
    cypher.append("unwind $records AS record")
    cypher.append("match (source:" + getLocalPart(row.dom) + " { `" + mappings[getLocalPart(row.dom)]["@uniqueId"] + "`: record.`" + mappings[getLocalPart(row.rel)]["@from"] + "`} )")
    cypher.append("match (target:" + getLocalPart(row.ran) + " { `" + mappings[getLocalPart(row.ran)]["@uniqueId"] + "`: record.`" + mappings[getLocalPart(row.rel)]["@to"] + "`} )")
    cypher.append("merge (source)-[r:`"+ getLocalPart(row.rel) +"`]->(target)")
    cypher.append("return count(*) as total") 
    cypher_import[getLocalPart(row.rel)] = ' \n'.join(cypher)


  nscount = 0
  mapping_export_cypher = []
  
  for ns in export_ns:
    mapping_export_cypher.append("call n10s.nsprefixes.add('ns" + str(nscount) + "','" + ns + "');")
    nscount+=1

  for k in export_mappings.keys():
    mapping_export_cypher.append("call n10s.mapping.add('" + export_mappings[k] + "','" + k + "');")

  return cypher_import ,  mapping_export_cypher




In [ ]:
cypher_import , mapping_defs = getLoadersFromOnto("https://raw.githubusercontent.com/jbarrasa/goingmeta/main/session5/ontos/rail.ttl","turtle",railMappings)

print("#LOADERS:\n\n")
for q in cypher_import.keys():
  print(q + ": \n\nfile: " + railMappings[q]["@fileName"] + "\n\n"+ cypher_import[q] + "\n\n")

print("#EXPORT MAPPINGS (for RDF API):\n\n")
for md in mapping_defs:
  print(md)

In [ ]:
# Utility function to write to Neo4j in batch mode.

def insert_data(session, query, frame, batch_size = 500):

    total = 0
    batch = 0
    start = time.time()
    result = None
    
    while batch * batch_size < len(frame):
        res = session.write_transaction( lambda tx: tx.run(query,
                      parameters = {'records': frame[batch*batch_size:(batch+1)*batch_size].to_dict('records')}).data())

        total += res[0]['total']
        batch += 1
        result = {"total":total, 
                  "batches":batch, 
                  "time":time.time()-start}
        print(result)
        
    return result

In [ ]:
import pandas as pd
from neo4j import GraphDatabase, basic_auth

driver = GraphDatabase.driver(
  "bolt://3.84.57.61:7687",
  auth=basic_auth("neo4j", "south-kettles-harmony"))

session = driver.session(database="neo4j")

cypher_import , mapping_defs = getLoadersFromOnto("https://raw.githubusercontent.com/jbarrasa/goingmeta/main/session5/ontos/rail.ttl","turtle",railMappings)

for q in cypher_import.keys():
  print("about to import " + q + " from file " + railMappings[q]["@fileName"])
  df = pd.read_csv(railMappings[q]["@fileName"])
  result = insert_data(session, cypher_import[q], df, batch_size = 300) 
  print(result)

for md in mapping_defs:
  session.run(md)
  